# Statistical learning

In this project we briefly discuss a few packages to do statistical learning in Julia.

*Aside:* I'm not an expert in this field, so if some of you has more insights or better tricks / setups to try, I'm happy for any comments!

## Stheno - Gaussian process regression

This first section considers Gaussian process regression and inference as implemented in the [Stheno](https://juliagaussianprocesses.github.io/Stheno.jl) package.

In [ ]:
using AbstractGPs
using Stheno
using Plots
using LinearAlgebra

In [ ]:
# Generate some synthetic data
N = 50
xdata = 2π * rand(N)
y = sin.(xdata) + 0.05*randn(N);

In [ ]:
# Setup zero-mean Gaussian Process with a squared exponential kernel σ² exp(-||x-x'||^2 / 2l²)
l  = 0.4
σ² = 0.6
f  = @gppp let
    # Build a Gaussian Process Programming Problem, but only with one GP
    # (could add multiple GPs here with parameters that depend on each other etc.)
    f1 = σ² * stretch(GP(SEKernel()), 1 / l)
end

# Sample from the GP f1 (with some IID observation noise)
# which is put on the diagonal of the Kernel
const x = GPPPInput(:f1, xdata)
σ²ₙ = 0.05
fx = f(x, σ²ₙ)

# Compute posterior over f given data y
f_posterior = posterior(fx, y)

# Plot data
plt = plot();
scatter!(plt, x.x, y; color=:red, label="");

# Plot posterior
x_plot = range(-0.5, 6.7; length=1000);
xp = GPPPInput(:f1, x_plot)
plot!(plt, x_plot, f_posterior(xp); label="", color=:blue,
      fillalpha=0.2, linewidth=2) 
plot!(
    plt, x_plot, rand(f_posterior(xp, 1e-9), 10);
    samples=10, markersize=1, alpha=0.3, label="", color=:blue,
)

In [ ]:
# Hyperparameter optimisation
using Optim
using Zygote

# Unpack hyperparameters (given in log scale)
# Fudge term to avoid nasty zeros ...
unpack(θ) = exp.(θ) .+ 1e-6

# nlml = negative log marginal likelihood (of θ)
function nlml(θ)
    l, σ², σ²ₙ = unpack(θ)
    f = @gppp let
        f1 = σ² * stretch(GP(SEKernel()), 1 / l)
    end
    -logpdf(f(x, σ²ₙ), y)
end

# Optimise using BFGS with adjoint-mode autodiff gradient on nlml
θ0 = randn(3);
results = Optim.optimize(nlml, nlml', θ0, BFGS(); inplace=false)
l_opt, σ²_opt, σ²ₙ_opt = unpack(results.minimizer)

@show σ²_opt
@show l_opt
@show σ²ₙ_opt

# Optimal GP and posterior
f_opt = @gppp let
    f1 = σ²_opt * stretch(GP(SEKernel()), 1 / l_opt)
end
f_posterior_opt = posterior(f(x, σ²ₙ_opt), y)

# Add to plot ...
plot!(plt, x_plot, f_posterior_opt(xp); label="", color=:orange,
      fillalpha=0.2, linewidth=2)
plot!(
    plt, x_plot, rand(f_posterior(xp, 1e-9), 10);
    samples=10, markersize=1, alpha=0.3, label="", color=:orange,
)

## Flux - The ML library that doesn't make you tensor

[Flux](https://fluxml.ai/) has become the de-facto standard for neural-network-based statistical learning in Julia. A good starting point for any NN-based learning problem is the [Flux model zoo](https://github.com/FluxML/model-zoo/).

What we will do here is handwriting recognition based on the `MNIST` data set.

In [ ]:
using Flux
using CUDA
using Plots
using PyCall
using MLDatasets: MNIST
using MLDataPattern
using Flux.Data: DataLoader
using FluxTraining

# "Accept" terms when downloading MNIST image data
ENV["DATADEPS_ALWAYS_ACCEPT"] = true;

In [ ]:
# Load handwritten digits dataset
x_data = reshape(MNIST.traintensor(Float32), 28, 28, 1, :)
y_data = Flux.onehotbatch(MNIST.trainlabels(), 0:9)

# Split into training, validation, testing
traindata, valdata, testdata = splitobs((x_data, y_data), at=(0.9, 0.05))

# Show sizes
@show size(traindata[1]) size(valdata[1]) size(testdata[1])
@show size(traindata[2]) size(valdata[2]) size(testdata[2]);

Now let us get a quick idea what we are dealing with. We plot a few of the images:

In [ ]:
plots = map(1:25) do i
    p = plot(Gray.(1 .- testdata[1][:, :, 1, i])'; yaxis=nothing, xaxis=nothing)
    label = Flux.onecold(testdata[2][:, i], 0:9)
    title!(p, string(label), titlefontsize=8)
end
plot(plots...)

Ok ... so a bunch of handwritten numbers, labelled with the appropriate image.

Next we define a convolutional neural network to be trained. Each Image has 28x28 pixels, that's the input size. Output is the 10 probabilities for being one of the numbers 0 to 9.

For more details on the layers and models, see
- https://fluxml.ai/Flux.jl/stable/models/layers/
- https://fluxml.ai/Flux.jl/stable/models/nnlib/

In [ ]:
model = Chain(
    Conv((3, 3), 1 => 6, relu),   # Convolution layer, 6 convolution kernels of size 3x3, relu activation
    AdaptiveMaxPool((13, 13)),    # In each layer keep only the largest of 2x2 windows
    Conv((3, 3), 6 => 16, relu),  # Another convolution ...
    AdaptiveMaxPool((5, 5)),      # ... and another pooling layer
    flatten,                      # Flatten all the data
    Dense(400, 100, relu),        # down to 100 outputs by a dense layer
    Dense(100, 84, relu),         # to 84
    Dense(84, 10),                # to 10
    softmax
)

Now we train (This will take a while ...)

In [ ]:
loss = Flux.Losses.crossentropy
opt  = Flux.Optimise.ADAM()

train_iterator = DataLoader(traindata, batchsize=250, shuffle=true)
valid_iterator = DataLoader(valdata,   batchsize=250, shuffle=true)
learner        = Learner(model, (train_iterator, valid_iterator),
                         opt, loss, Metrics(accuracy), ToGPU())  # Use GPU if available

nepochs = 10
FluxTraining.fit!(learner, nepochs)

To verify our training did not overfit, we check the learning rate comparing the training and validation loss over the learning epochs:

In [ ]:
training_loss   = learner.cbstate.metricsepoch[TrainingPhase()][:Loss].values
validation_loss = learner.cbstate.metricsepoch[ValidationPhase()][:Loss].values

p = plot(training_loss, label="Training loss")
p = plot!(p, validation_loss, label="Validation loss")

As we see the rates for training and validation are nicely paralell, so we did not yet overfit.

Let's assume we are happy with our results (we could do another 10 epochs of training if we we were not). After all training is done we check how good our model is with the test data:

In [ ]:
# Evaluate the model using the test data
test_pred = model(testdata[1])

@show loss(test_pred, testdata[2])
@show accuracy(test_pred, testdata[2]);

Not too bad, let's see what this means in practice:

In [ ]:
plots = map(1:25) do i
    p = plot(Gray.(1 .- testdata[1][:, :, 1, i])'; yaxis=nothing, xaxis=nothing)
    prediction = findmax(vec(model(testdata[1][:, :, :, i:i])))
    title!(p, "$(prediction[2]-1) -> $(round(100prediction[1], digits=1))%",
    titlefontsize=8)
end
plot(plots...)